In [ ]:
import numpy as np
from scipy.stats import ortho_group
import sklearn.linear_model as lin
import time
from mpc import *
import matplotlib.pyplot as plt

In [ ]:
def subspace_basis(t, d):
    U = ortho_group.rvs(d)
    return U[:t]

# B is t x d orthogonal basis vectors, project it out of vector v
def subspace_project(B, v):
    for i in xrange(len(B)):
        v = v - np.dot(v, B[i]) * B[i]
    return v

# scale covariance down
def scaled_normal(d):
    return np.random.normal(scale = 1.0 / np.sqrt(d), size = d)

In [ ]:
# random n1 rows outside small subspace
# basis of subspace, blown up to be quite large
# other n2 random rows that are also large
def bad_instance(t, d, n1, n2, size):
    A1 = subspace_basis(t, d)
    A2 = np.asarray([subspace_project(A1, scaled_normal(d)) for i in xrange(n1)])
    A3 = np.asarray([scaled_normal(d) for i in xrange(n2)])
    A = np.vstack((A2, A1 * size, A3 * size))
    np.random.shuffle(A)
    return A

In [ ]:
def row_norm_noise(A, xtrue, stdev):
    (n, d) = np.shape(A)
    row_norms = np.asarray([np.linalg.norm(A[i]) for i in xrange(n)])
    noise = np.dot(np.diag(row_norms), np.random.normal(scale = stdev, size = n))
    return np.dot(A, xtrue) + noise

In [ ]:
# MPC parameters
degree = 10 # degree of rational approximation
jldirs = 5 # number of JL directions used
niters = 8 # number of iterations of solver
alpha = 1.0 # step size of solver
coeffs = np.asarray(ratapprox(degree))
reg = lin.Ridge(alpha=0.0, solver='lsqr', tol=1e-5)

# Naive and MPC solver, l2 squared error, d = 100

In [ ]:
(t, d, n2, size) = (10, 100, 50, 100)
squared_errors = np.zeros((10, 15))
for i in xrange(10): # different values of n1
    print i
    n1 = 400 * (i + 1)
    A = bad_instance(t, d, n1, n2, size)
    for j in xrange(15):
        xtrue = np.random.normal(size = d)
        b = row_norm_noise(A, xtrue, 0.1)
        x = reg.fit(A, b).coef_
        squared_errors[i][j] = (np.linalg.norm(xtrue - x) ** 2)

In [ ]:
(t, d, n2, size) = (10, 100, 50, 100)
squared_errors_mpc = np.zeros((10, 15))
for i in xrange(10): # different values of n1
    print i
    n1 = 400 * (i + 1)
    A = bad_instance(t, d, n1, n2, size)
    (w, total_mv) = mpc_ideal_niters(A, 500, 1.0, niters, jldirs, coeffs)
    for j in xrange(15):
        xtrue = np.random.normal(size = d)
        b = row_norm_noise(A, xtrue, 0.1)
        x = reg.fit(WhalfA(A, w), np.sqrt(w) * b).coef_
        squared_errors_mpc[i][j] = (np.linalg.norm(xtrue - x) ** 2)

In [ ]:
means_squared_errors = np.asarray([np.mean(squared_errors[i]) for i in xrange(10)])
means_squared_errors_mpc = np.asarray([np.mean(squared_errors_mpc[i]) for i in xrange(10)])
stdevs_squared_errors = np.asarray([np.std(squared_errors[i]) for i in xrange(10)])
stdevs_squared_errors_mpc = np.asarray([np.std(squared_errors_mpc[i]) for i in xrange(10)])

In [ ]:
plt.figure()
plt.errorbar(x=np.arange(400, 4400, 400), y=means_squared_errors, yerr=stdevs_squared_errors)
plt.errorbar(x=np.arange(400, 4400, 400), y=means_squared_errors_mpc, yerr=stdevs_squared_errors_mpc)
plt.xticks(np.arange(400, 4400, 400))
plt.title("Mean squared errors, d = 100")
plt.xlabel("Clean data point count")
plt.ylabel("Mean squared error")
plt.show()

# Naive and MPC solver, l2 squared error, d = 200

In [ ]:
(t, d, n2, size) = (10, 200, 100, 200)
squared_errors_2 = np.zeros((10, 15))
for i in xrange(10): # different values of n1
    print i
    n1 = 800 * (i + 1)
    A = bad_instance(t, d, n1, n2, size)
    for j in xrange(15):
        xtrue = np.random.normal(size = d)
        b = row_norm_noise(A, xtrue, 0.1)
        x = reg.fit(A, b).coef_
        squared_errors_2[i][j] = (np.linalg.norm(xtrue - x) ** 2)

In [ ]:
(t, d, n2, size) = (10, 200, 100, 200)
squared_errors_mpc_2 = np.zeros((10, 15))
for i in xrange(10): # different values of n1
    print i
    n1 = 800 * (i + 1)
    A = bad_instance(t, d, n1, n2, size)
    (w, total_mv) = mpc_ideal_niters(A, 1000, 1.0, niters, jldirs, coeffs)
    for j in xrange(15):
        xtrue = np.random.normal(size = d)
        b = row_norm_noise(A, xtrue, 0.1)
        x = reg.fit(WhalfA(A, w), np.sqrt(w) * b).coef_
        squared_errors_mpc_2[i][j] = (np.linalg.norm(xtrue - x) ** 2)

In [ ]:
means_squared_errors_2 = np.asarray([np.mean(squared_errors_2[i]) for i in xrange(10)])
means_squared_errors_mpc_2 = np.asarray([np.mean(squared_errors_mpc_2[i]) for i in xrange(10)])
stdevs_squared_errors_2 = np.asarray([np.std(squared_errors_2[i]) for i in xrange(10)])
stdevs_squared_errors_mpc_2 = np.asarray([np.std(squared_errors_mpc_2[i]) for i in xrange(10)])

In [ ]:
plt.figure()
plt.errorbar(x=np.arange(800, 8800, 800), y=means_squared_errors_2, yerr=stdevs_squared_errors_2)
plt.errorbar(x=np.arange(800, 8800, 800), y=means_squared_errors_mpc_2, yerr=stdevs_squared_errors_mpc_2)
plt.xticks(np.arange(800, 8800, 800))
plt.title("Mean squared errors, d = 200")
plt.xlabel("Clean data point count")
plt.ylabel("Mean squared error")
plt.show()

# Naive and MPC solver, l2 squared error, d = 300

In [ ]:
(t, d, n2, size) = (10, 300, 150, 300)
squared_errors_3 = np.zeros((10, 15))
for i in xrange(10): # different values of n1
    print i
    n1 = 1200 * (i + 1)
    A = bad_instance(t, d, n1, n2, size)
    for j in xrange(15):
        xtrue = np.random.normal(size = d)
        b = row_norm_noise(A, xtrue, 0.1)
        x = reg.fit(A, b).coef_
        squared_errors_3[i][j] = (np.linalg.norm(xtrue - x) ** 2)

In [ ]:
(t, d, n2, size) = (10, 300, 150, 300)
squared_errors_mpc_3 = np.zeros((10, 15))
for i in xrange(10): # different values of n1
    print i
    n1 = 1200 * (i + 1)
    A = bad_instance(t, d, n1, n2, size)
    (w, total_mv) = mpc_ideal_niters(A, 1500, 1.0, niters, jldirs, coeffs)
    for j in xrange(15):
        xtrue = np.random.normal(size = d)
        b = row_norm_noise(A, xtrue, 0.1)
        x = reg.fit(WhalfA(A, w), np.sqrt(w) * b).coef_
        squared_errors_mpc_3[i][j] = (np.linalg.norm(xtrue - x) ** 2)

In [ ]:
means_squared_errors_3 = np.asarray([np.mean(squared_errors_3[i]) for i in xrange(10)])
means_squared_errors_mpc_3 = np.asarray([np.mean(squared_errors_mpc_3[i]) for i in xrange(10)])
stdevs_squared_errors_3 = np.asarray([np.std(squared_errors_3[i]) for i in xrange(10)])
stdevs_squared_errors_mpc_3 = np.asarray([np.std(squared_errors_mpc_3[i]) for i in xrange(10)])

In [ ]:
plt.figure()
plt.errorbar(x=np.arange(1200, 13200, 1200), y=means_squared_errors_3, yerr=stdevs_squared_errors_3)
plt.errorbar(x=np.arange(1200, 13200, 1200), y=means_squared_errors_mpc_3, yerr=stdevs_squared_errors_mpc_3)
plt.xticks(np.arange(1200, 13200, 1200))
plt.title("Mean squared errors, d = 300")
plt.xlabel("Clean data point count")
plt.ylabel("Mean squared error")
plt.show()